In [ ]:
# CrossFit 코칭 애플리케이션 - Google Colab 버전
# Google Drive 연동 및 API 키 설정

import os
import sys

# Google Colab 환경 확인
try:
    import google.colab
    IN_COLAB = True
    print("🌟 Google Colab 환경에서 실행 중입니다.")
except ImportError:
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중입니다.")

if IN_COLAB:
    # 1. Google Drive 마운트
    from google.colab import drive, userdata
    drive.mount('/content/drive')
    
    # 2. 프로젝트 경로 설정
    project_path = "/content/drive/MyDrive/2조_3rd_프로젝트"
    if os.path.exists(project_path):
        os.chdir(project_path)
        print(f"📁 프로젝트 디렉토리로 이동: {project_path}")
    else:
        print(f"❌ 프로젝트 디렉토리를 찾을 수 없습니다: {project_path}")
        print("💡 Google Drive에서 프로젝트 폴더를 확인해주세요.")
    
    # 3. API 키 설정
    try:
        os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
        os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
        print("🔑 API 키가 성공적으로 설정되었습니다.")
    except Exception as e:
        print(f"⚠️  API 키 설정 실패: {e}")
        print("💡 Colab의 '비밀' 섹션에서 LANGCHAIN_API_KEY와 OPENAI_API_KEY를 설정해주세요.")
    
    # 4. 시스템 패스 추가
    if project_path not in sys.path:
        sys.path.append(project_path)

else:
    # 로컬 환경에서의 설정
    sys.path.append(os.path.join(os.getcwd(), '..'))

print("🏋️ CrossFit 코칭 애플리케이션 시작")
print(f"📁 현재 작업 디렉토리: {os.getcwd()}")

In [ ]:
# Google Colab용 패키지 설치 및 모듈 import

if IN_COLAB:
    # Google Colab에서 필요한 패키지 설치
    print("📦 필요한 패키지를 설치합니다...")
    !pip install -q gradio
    !pip install -q openai
    !pip install -q langchain
    !pip install -q langchain-openai
    !pip install -q langchain-community
    !pip install -q chromadb
    !pip install -q pypdf
    !pip install -q pandas
    !pip install -q psutil
    print("✅ 패키지 설치 완료")

# 필요한 라이브러리 및 모듈 import
import gradio as gr
import sqlite3
import json
from datetime import datetime
from typing import Dict, Any, List, Optional
import pandas as pd

# src 모듈들 import
try:
    from src.config import config, DB_PATH, CHROMA_DIR, CHROMA_BACKUP_DIR, PDF_GUIDE_DIR
    from src.utils import NetworkUtils, StateUtils
    from src.models import UserModel, QAModel, VectorDBModel
    from src.main import CrossFitApp
    print("✅ 모든 src 모듈이 성공적으로 로드되었습니다.")
except ImportError as e:
    print(f"⚠️  모듈 로드 오류: {e}")
    if IN_COLAB:
        print("📌 Google Drive에서 프로젝트 구조를 확인해주세요.")
        print("💡 프로젝트 폴더가 '/content/drive/MyDrive/2조_3rd_프로젝트'에 있는지 확인하세요.")
    else:
        print("📌 src 폴더 구조를 확인해주세요.")

## 🎯 애플리케이션 초기화 및 설정

## 🌟 Google Colab 전용 설정 및 확인

In [ ]:
# Google Colab 환경 설정 및 상태 확인

def check_colab_setup():
    """Google Colab 설정 상태를 확인합니다"""
    print("🔍 Google Colab 설정 확인 중...")
    
    if not IN_COLAB:
        print("⚠️  이 셀은 Google Colab에서만 실행됩니다.")
        return
    
    # 1. Google Drive 연결 확인
    print("\n📁 Google Drive 연결 상태:")
    if os.path.exists("/content/drive"):
        print("   ✅ Google Drive가 마운트되었습니다")
        
        # 프로젝트 폴더 확인
        project_path = "/content/drive/MyDrive/2조_3rd_프로젝트"
        if os.path.exists(project_path):
            print(f"   ✅ 프로젝트 폴더 발견: {project_path}")
            
            # 폴더 내용 확인
            contents = os.listdir(project_path)
            print(f"   📂 폴더 내용: {len(contents)}개 항목")
            for item in contents[:5]:  # 처음 5개만 표시
                print(f"      - {item}")
            if len(contents) > 5:
                print(f"      ... 및 {len(contents) - 5}개 추가 항목")
        else:
            print(f"   ❌ 프로젝트 폴더를 찾을 수 없습니다: {project_path}")
            print("   💡 Google Drive에서 프로젝트를 업로드했는지 확인하세요")
    else:
        print("   ❌ Google Drive가 마운트되지 않았습니다")
        print("   💡 첫 번째 셀을 다시 실행하여 Drive를 마운트하세요")
    
    # 2. API 키 확인
    print("\n🔑 API 키 설정 상태:")
    try:
        from google.colab import userdata
        
        # OPENAI_API_KEY 확인
        try:
            openai_key = userdata.get('OPENAI_API_KEY')
            if openai_key:
                print(f"   ✅ OPENAI_API_KEY: {openai_key[:8]}...{openai_key[-4:]}")
            else:
                print("   ❌ OPENAI_API_KEY: 설정되지 않음")
        except Exception:
            print("   ❌ OPENAI_API_KEY: 접근 실패")
        
        # LANGCHAIN_API_KEY 확인 (선택사항)
        try:
            langchain_key = userdata.get('LANGCHAIN_API_KEY')
            if langchain_key:
                print(f"   ✅ LANGCHAIN_API_KEY: {langchain_key[:8]}...{langchain_key[-4:]}")
            else:
                print("   ⚠️  LANGCHAIN_API_KEY: 설정되지 않음 (선택사항)")
        except Exception:
            print("   ⚠️  LANGCHAIN_API_KEY: 접근 실패 (선택사항)")
            
    except ImportError:
        print("   ❌ userdata 모듈을 불러올 수 없습니다")
    
    # 3. 패키지 설치 상태 확인
    print("\n📦 필수 패키지 설치 상태:")
    required_packages = [
        "gradio", "openai", "langchain", "langchain-openai", 
        "langchain-community", "chromadb", "pypdf"
    ]
    
    for package in required_packages:
        try:
            __import__(package.replace('-', '_'))
            print(f"   ✅ {package}: 설치됨")
        except ImportError:
            print(f"   ❌ {package}: 설치 필요")
    
    # 4. 설정 완료 여부 종합 판단
    print("\n🎯 설정 완료 상태:")
    drive_ok = os.path.exists("/content/drive")
    project_ok = os.path.exists("/content/drive/MyDrive/2조_3rd_프로젝트")
    api_ok = 'OPENAI_API_KEY' in os.environ
    
    if drive_ok and project_ok and api_ok:
        print("   🎉 모든 설정이 완료되었습니다! 애플리케이션을 실행할 수 있습니다.")
    else:
        print("   ⚠️  일부 설정이 미완료되었습니다:")
        if not drive_ok:
            print("      - Google Drive 마운트 필요")
        if not project_ok:
            print("      - 프로젝트 폴더 업로드 필요")
        if not api_ok:
            print("      - OpenAI API 키 설정 필요")
        print("   💡 위의 문제를 해결한 후 다시 시도하세요.")

# Google Colab 설정 확인 실행
check_colab_setup()

In [ ]:
# Google Colab용 애플리케이션 초기화

def initialize_colab_environment():
    """Google Colab 환경 초기화"""
    if IN_COLAB:
        # 데이터 디렉토리가 없으면 생성
        data_dirs = ['data', 'data/sqlite_db', 'data/chroma_db', 'data/chroma_db_backups', 
                    'data/raw', 'data/raw/crossfit_guide', 'data/processed']
        
        for dir_path in data_dirs:
            if not os.path.exists(dir_path):
                os.makedirs(dir_path, exist_ok=True)
                print(f"📁 디렉토리 생성: {dir_path}")
        
        # 환경 변수 재확인
        if 'OPENAI_API_KEY' not in os.environ or not os.environ.get('OPENAI_API_KEY'):
            print("⚠️  OPENAI_API_KEY가 설정되지 않았습니다.")
            print("💡 Colab 왼쪽 사이드바의 '🔑' 아이콘을 클릭하여 API 키를 추가하세요.")
        else:
            print("✅ OPENAI_API_KEY 설정 확인됨")

# Colab 환경 초기화
initialize_colab_environment()

# 애플리케이션 인스턴스 생성
try:
    app = CrossFitApp()
    print("🚀 CrossFit 애플리케이션이 성공적으로 초기화되었습니다!")
    print(f"📊 사용자 모델: {type(app.user_model).__name__}")
    print(f"🤖 QA 모델: {type(app.qa_model).__name__}")
    print(f"🗄️  VectorDB 모델: {type(app.vector_db_model).__name__}")
    
    if IN_COLAB:
        print("🌟 Google Colab 환경에서 정상 실행 중")
        
except Exception as e:
    print(f"❌ 애플리케이션 초기화 실패: {e}")
    if IN_COLAB:
        print("💡 문제 해결 방법:")
        print("   1. Google Drive 마운트 확인")
        print("   2. 프로젝트 폴더 경로 확인")
        print("   3. API 키 설정 확인")
        print("   4. 인터넷 연결 상태 확인")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 현재 src 코드 기반 QA 시스템 테스트
def test_qa_system():
    """QA 시스템 기능 테스트"""
    print("🧪 QA 시스템 테스트 시작...")
    
    # 1. 기본 용어 검색 테스트
    print("\n1️⃣ 기본 용어 검색 테스트:")
    glossary_terms = app.app_state.get("glossary", [])
    print(f"   📚 등록된 용어 수: {len(glossary_terms)}")
    
    if glossary_terms:
        print("   📖 등록된 주요 용어들:")
        for i, term in enumerate(glossary_terms[:5]):  # 처음 5개만 출력
            print(f"      {i+1}. {term.get('term', 'N/A')}: {term.get('definition', 'N/A')[:50]}...")
    
    # 2. AI 검색 기능 테스트 (OpenAI 연동)
    print("\n2️⃣ AI 검색 기능 테스트:")
    test_queries = ["스내치", "WOD", "크로스핏 기본 운동"]
    
    for query in test_queries:
        try:
            result = app.qa_model.search_glossary_with_ai(query, app.app_state["glossary"])
            print(f"   🔍 '{query}' 검색 결과:")
            if result and len(result) > 0:
                print(f"      ✅ 성공: {result[:100]}...")
            else:
                print(f"      ⚠️  결과 없음")
        except Exception as e:
            print(f"      ❌ 오류: {e}")
    
    # 3. 벡터 DB 연결 상태 확인
    print("\n3️⃣ VectorDB 연결 상태:")
    try:
        if app.vector_db_model and hasattr(app.vector_db_model, 'vectorstore'):
            if app.vector_db_model.vectorstore is not None:
                print("   ✅ VectorDB 연결됨")
            else:
                print("   ⚠️  VectorDB 연결 안됨")
        else:
            print("   ⚠️  VectorDB 모델 초기화 안됨")
    except Exception as e:
        print(f"   ❌ VectorDB 상태 확인 실패: {e}")
    
    # 4. 챗봇 컨트롤러 테스트
    print("\n4️⃣ 챗봇 컨트롤러 테스트:")
    try:
        test_message = "크로스핏이 뭔가요?"
        # 로그인 상태 시뮬레이션
        app.app_state["user_session"]["auth"] = True
        app.app_state["user_session"]["user_id"] = "test_user"
        
        response = app.chatbot_controller.chat_response(
            message=test_message,
            history=[],
            app_state=app.app_state
        )
        print(f"   🤖 챗봇 응답 테스트:")
        print(f"      질문: {test_message}")
        if isinstance(response, tuple) and len(response) >= 2:
            print(f"      응답: {response[1][:100] if response[1] else 'None'}...")
        else:
            print(f"      응답: {str(response)[:100]}...")
            
    except Exception as e:
        print(f"   ❌ 챗봇 테스트 실패: {e}")

# 테스트 실행
test_qa_system()

In [ ]:
# 사용자 인증 시스템 테스트
def test_auth_system():
    """사용자 인증 시스템 테스트"""
    print("🔐 사용자 인증 시스템 테스트...")
    
    # 테스트 사용자 정보
    test_email = "test@crossfit.com"
    test_password = "testpass123"
    
    try:
        # 1. 회원가입 테스트
        print("\n1️⃣ 회원가입 테스트:")
        signup_result = app.auth_controller.register(test_email, test_password, test_password)
        print(f"   결과: {signup_result}")
        
        # 2. 로그인 테스트
        print("\n2️⃣ 로그인 테스트:")
        login_result = app.auth_controller.login(test_email, test_password)
        print(f"   결과: {login_result}")
        
        # 3. 세션 상태 확인
        print("\n3️⃣ 세션 상태 확인:")
        session = app.app_state.get("user_session", {})
        print(f"   인증 상태: {session.get('auth', False)}")
        print(f"   사용자 ID: {session.get('user_id', 'None')}")
        print(f"   이메일: {session.get('email', 'None')}")
        
    except Exception as e:
        print(f"   ❌ 인증 시스템 테스트 실패: {e}")

test_auth_system()

In [ ]:
# 개인 맞춤 추천 시스템 테스트
def test_recommendation_system():
    """개인 맞춤 추천 시스템 테스트"""
    print("🎯 개인 맞춤 추천 시스템 테스트...")
    
    try:
        # 사용자 인증 상태 확인
        if not app.app_state["user_session"]["auth"]:
            app.app_state["user_session"]["auth"] = True
            app.app_state["user_session"]["user_id"] = "test_user"
            print("   🔐 테스트용 로그인 상태 설정")
        
        # 추천 시스템 테스트
        test_params = {
            "level": "초급",
            "goal": "체중감량",
            "freq": "3회",
            "gear": ["케틀벨", "덤벨"]
        }
        
        print(f"\n📋 테스트 파라미터:")
        for key, value in test_params.items():
            print(f"   {key}: {value}")
        
        recommendation_result = app.recommendation_controller.generate_workout_plan(
            level=test_params["level"],
            goal=test_params["goal"], 
            freq=test_params["freq"],
            gear=test_params["gear"]
        )
        
        print(f"\n🎯 추천 결과:")
        if isinstance(recommendation_result, tuple) and len(recommendation_result) >= 2:
            result_text, result_data = recommendation_result
            print(f"   텍스트 결과: {result_text[:200]}..." if result_text else "   텍스트 없음")
            if result_data:
                print(f"   데이터 결과: {str(result_data)[:200]}...")
        else:
            print(f"   결과: {str(recommendation_result)[:200]}...")
            
    except Exception as e:
        print(f"   ❌ 추천 시스템 테스트 실패: {e}")

test_recommendation_system()

In [ ]:
# 주제별 자동 쿼리 생성 및 응답 시스템 테스트
def test_topic_system():
    """주제별 시스템 테스트"""
    print("📚 주제별 자동 쿼리 및 응답 시스템 테스트...")
    
    # 지원하는 주제들
    topics = ["용어/규칙/운동법", "식단/회복", "인증/챌린지 안내", "멘토링(초보 심리/동기)"]
    
    try:
        for topic in topics:
            print(f"\n🎯 주제: {topic}")
            
            # 1. 자동 쿼리 생성 테스트
            auto_query = app.qa_model.generate_topic_query(topic)
            print(f"   📝 생성된 쿼리: {auto_query}")
            
            # 2. 주제별 응답 생성 테스트
            try:
                topic_response = app.topic_controller.generate_topic_response(topic)
                if isinstance(topic_response, tuple) and len(topic_response) >= 2:
                    response_text = topic_response[1]
                    print(f"   🤖 AI 응답: {response_text[:150]}..." if response_text else "   응답 없음")
                else:
                    print(f"   🤖 응답: {str(topic_response)[:150]}...")
            except Exception as e:
                print(f"   ⚠️  응답 생성 실패: {e}")
                
    except Exception as e:
        print(f"❌ 주제별 시스템 테스트 실패: {e}")

test_topic_system()

⚡ 기존 VectorDB가 이미 존재합니다. batch/bulk 임베딩 건너뜀.


In [ ]:
# 관리자 기능 테스트 (VectorDB 관리)
def test_admin_system():
    """관리자 시스템 테스트"""
    print("🛠️ 관리자 VectorDB 관리 시스템 테스트...")
    
    try:
        # 1. VectorDB 백업 목록 조회
        print("\n1️⃣ VectorDB 백업 목록:")
        backup_rows = app.admin_controller.get_all_db_rows()
        if isinstance(backup_rows, list) and len(backup_rows) > 0:
            print(f"   📦 발견된 백업 파일 수: {len(backup_rows)}")
            for i, row in enumerate(backup_rows[:3]):  # 처음 3개만 표시
                if isinstance(row, list) and len(row) >= 5:
                    print(f"      {i+1}. 버전: {row[1]}, 파일명: {row[2]}, 크기: {row[3]}")
        else:
            print("   📦 백업 파일이 없습니다.")
        
        # 2. 현재 VectorDB 상태 확인
        print("\n2️⃣ 현재 VectorDB 상태:")
        if hasattr(app.vector_db_model, 'vectorstore') and app.vector_db_model.vectorstore:
            print("   ✅ VectorDB가 활성화되어 있습니다.")
        else:
            print("   ⚠️  VectorDB가 비활성화되어 있습니다.")
            
        # 3. 테스트 백업 생성 (실제로는 하지 않고 시뮬레이션만)
        print("\n3️⃣ 백업 생성 기능:")
        print("   📝 백업 생성은 실제 파일을 생성하므로 테스트에서는 스킵합니다.")
        print("   💡 실제 사용 시: app.admin_controller.create_backup('테스트백업')")
        
    except Exception as e:
        print(f"❌ 관리자 시스템 테스트 실패: {e}")

test_admin_system()

/tmp/ipython-input-3957547275.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


VectorDB 로딩 완료!


In [ ]:
# 전체 시스템 통합 테스트
def comprehensive_system_test():
    """전체 시스템 통합 테스트"""
    print("🔄 전체 시스템 통합 테스트...")
    
    try:
        # 1. 시스템 상태 종합 점검
        print("\n📊 시스템 상태 종합:")
        
        # 모델 상태 확인
        models_status = {
            "UserModel": app.user_model is not None,
            "QAModel": app.qa_model is not None, 
            "VectorDBModel": app.vector_db_model is not None
        }
        
        # 컨트롤러 상태 확인
        controllers_status = {
            "AuthController": app.auth_controller is not None,
            "ChatbotController": app.chatbot_controller is not None,
            "VideoController": app.video_controller is not None,
            "RecommendationController": app.recommendation_controller is not None,
            "TopicController": app.topic_controller is not None,
            "AdminController": app.admin_controller is not None
        }
        
        print("   🔧 모델 상태:")
        for model, status in models_status.items():
            print(f"      {'✅' if status else '❌'} {model}: {'OK' if status else 'FAILED'}")
            
        print("   🎮 컨트롤러 상태:")  
        for controller, status in controllers_status.items():
            print(f"      {'✅' if status else '❌'} {controller}: {'OK' if status else 'FAILED'}")
        
        # 2. 애플리케이션 상태 확인
        print("\n📱 애플리케이션 상태:")
        app_state_keys = ["user_session", "chat_history", "evidence_library", "glossary"]
        for key in app_state_keys:
            if key in app.app_state:
                if key == "glossary":
                    print(f"   ✅ {key}: {len(app.app_state[key])} 항목")
                elif key == "evidence_library": 
                    print(f"   ✅ {key}: {len(app.app_state[key])} 항목")
                elif key == "user_session":
                    auth_status = app.app_state[key].get("auth", False)
                    print(f"   ✅ {key}: 인증={'ON' if auth_status else 'OFF'}")
                else:
                    print(f"   ✅ {key}: OK")
            else:
                print(f"   ❌ {key}: MISSING")
        
        # 3. OpenAI 연동 상태 확인
        print("\n🤖 OpenAI 연동 상태:")
        if hasattr(app.qa_model, 'llm') and app.qa_model.llm:
            print("   ✅ ChatGPT 모델: 연결됨")
        else:
            print("   ⚠️  ChatGPT 모델: 연결 안됨 (API 키 확인 필요)")
            
        if hasattr(app.qa_model, 'embeddings') and app.qa_model.embeddings:
            print("   ✅ 임베딩 모델: 연결됨")  
        else:
            print("   ⚠️  임베딩 모델: 연결 안됨")
        
        # 4. 종합 평가
        print("\n🏆 종합 평가:")
        total_components = len(models_status) + len(controllers_status)
        working_components = sum(models_status.values()) + sum(controllers_status.values())
        success_rate = (working_components / total_components) * 100
        
        print(f"   📊 컴포넌트 가동률: {working_components}/{total_components} ({success_rate:.1f}%)")
        
        if success_rate >= 90:
            print("   🎉 시스템이 정상적으로 작동하고 있습니다!")
        elif success_rate >= 70:
            print("   ⚠️  일부 컴포넌트에 문제가 있지만 기본 기능은 동작합니다.")
        else:
            print("   🚨 시스템에 심각한 문제가 있습니다. 설정을 확인해주세요.")
            
    except Exception as e:
        print(f"❌ 통합 테스트 실패: {e}")

comprehensive_system_test()

/tmp/ipython-input-960068463.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


## 🖥️ Gradio 웹 애플리케이션 실행

### 현재 src 코드 기반 전체 Gradio 인터페이스 실행

In [ ]:
# Google Colab용 Gradio 애플리케이션 실행
def launch_colab_gradio_app():
    """Google Colab에서 Gradio 앱 실행"""
    
    try:
        print("🚀 Gradio 웹 애플리케이션 시작...")
        
        # 애플리케이션의 create_interface 메소드 호출
        interface = app.create_interface()
        
        print("✅ 인터페이스 생성 완료")
        print("📱 모든 기능이 포함된 완전한 CrossFit 코칭 시스템:")
        print("   - 🔐 사용자 인증 (로그인/회원가입)")
        print("   - 🤖 AI 챗봇 (OpenAI GPT-4o-mini)")
        print("   - 📚 용어 사전 및 지식 검색")
        print("   - 🎯 개인 맞춤 운동 추천")
        print("   - 📹 비디오 코칭 분석")
        print("   - 🛠️ 관리자 VectorDB 관리")
        
        # Google Colab에서 Gradio 앱 실행
        if IN_COLAB:
            print("🌟 Google Colab에서 공개 링크로 실행합니다...")
            interface.launch(
                share=True,             # Colab에서는 공개 링크 필요
                debug=True,             # 디버그 모드
                height=800,             # Colab 화면에 맞는 높이
                show_error=True         # 에러 표시
            )
        else:
            # 로컬 환경에서의 실행
            interface.launch(
                server_name="0.0.0.0",
                server_port=7860,
                share=False,
                inbrowser=True,
                debug=True
            )
        
    except Exception as e:
        print(f"❌ Gradio 앱 실행 실패: {e}")
        
        # 간단한 테스트 인터페이스 제공
        print("\n🔧 간단한 테스트 인터페이스를 생성합니다...")
        
        def simple_chat(message, history):
            """간단한 채팅 기능"""
            try:
                # 테스트용 로그인 상태 설정
                if not app.app_state["user_session"]["auth"]:
                    app.app_state["user_session"]["auth"] = True
                    app.app_state["user_session"]["user_id"] = "demo_user"
                
                result = app.chatbot_controller.chat_response(
                    message=message,
                    history=history,
                    app_state=app.app_state
                )
                
                if isinstance(result, tuple) and len(result) >= 2:
                    return result[1]
                return str(result)
                
            except Exception as e:
                return f"오류가 발생했습니다: {e}"
        
        # 간단한 ChatInterface 생성
        simple_interface = gr.ChatInterface(
            fn=simple_chat,
            title="🏋️ CrossFit 코칭 챗봇 (Colab 버전)",
            description="CrossFit에 대해 궁금한 것을 물어보세요! Google Colab에서 실행 중입니다.",
            theme=gr.themes.Soft(),
            examples=[
                "크로스핏이 무엇인가요?",
                "WOD가 뭔가요?",
                "스내치 동작을 설명해주세요",
                "초보자 운동 계획을 추천해주세요"
            ]
        )
        
        print("📱 간단한 테스트 인터페이스를 실행합니다...")
        
        if IN_COLAB:
            simple_interface.launch(share=True, debug=True, height=600)
        else:
            simple_interface.launch(share=False, debug=True)

# Google Colab용 실행
launch_colab_gradio_app()

## 📊 시스템 모니터링 및 디버깅

In [ ]:
# Google Colab용 시스템 모니터링 및 진단 도구
def colab_system_diagnostics():
    """Google Colab 환경에서 시스템 진단"""
    print("🔍 Google Colab 시스템 진단 시작...")
    
    try:
        # 1. Google Colab 환경 정보
        if IN_COLAB:
            print("\n🌟 Google Colab 환경 정보:")
            # GPU 사용 가능 여부 확인
            gpu_available = "GPU" if os.system("nvidia-smi > /dev/null 2>&1") == 0 else "CPU Only"
            print(f"   🖥️ 컴퓨팅 환경: {gpu_available}")
            
            # Google Drive 마운트 상태 확인
            drive_mounted = os.path.exists("/content/drive")
            print(f"   📁 Google Drive: {'✅ 마운트됨' if drive_mounted else '❌ 마운트 안됨'}")
            
            # 현재 디렉토리와 파일 확인
            print(f"   📂 현재 디렉토리: {os.getcwd()}")
            
        # 2. 파일 시스템 상태 확인
        print("\n📁 프로젝트 파일 시스템:")
        project_files = ["src", "data", "requirements.txt", "run.py", "README.md"]
        
        for file_path in project_files:
            if os.path.exists(file_path):
                if os.path.isdir(file_path):
                    items = len(os.listdir(file_path))
                    print(f"   ✅ {file_path}/: {items}개 항목")
                else:
                    size = os.path.getsize(file_path) // 1024
                    print(f"   ✅ {file_path}: {size}KB")
            else:
                print(f"   ❌ {file_path}: 없음")
        
        # 3. 환경 변수 및 API 키 확인
        print("\n? 환경 변수 상태:")
        api_keys = ["OPENAI_API_KEY", "LANGCHAIN_API_KEY"]
        for key in api_keys:
            if key in os.environ and os.environ.get(key):
                masked_key = os.environ[key][:8] + "..." + os.environ[key][-4:] if len(os.environ[key]) > 12 else "설정됨"
                print(f"   ✅ {key}: {masked_key}")
            else:
                print(f"   ❌ {key}: 설정되지 않음")
        
        # 4. Python 패키지 확인 (Colab용)
        if IN_COLAB:
            print("\n📦 주요 패키지 버전:")
            packages = ["gradio", "openai", "langchain", "chromadb", "pandas"]
            for package in packages:
                try:
                    import importlib
                    module = importlib.import_module(package)
                    version = getattr(module, '__version__', 'Unknown')
                    print(f"   ✅ {package}: {version}")
                except ImportError:
                    print(f"   ❌ {package}: 설치되지 않음")
        
        # 5. 메모리 사용량 (Colab은 제한적)
        try:
            import psutil
            print("\n💾 시스템 리소스:")
            memory = psutil.virtual_memory()
            print(f"   ? 메모리 사용률: {memory.percent}%")
            print(f"   💾 사용가능 메모리: {memory.available // (1024**2)}MB")
            
            if IN_COLAB:
                print("   ⚠️  Colab은 RAM 제한이 있습니다 (보통 12GB)")
        except ImportError:
            if IN_COLAB:
                print("\n💾 시스템 리소스:")
                print("   📦 psutil 패키지를 설치하여 메모리 정보를 확인하세요")
        
        # 6. 애플리케이션 상태 확인
        print("\n🎯 애플리케이션 상태:")
        try:
            if 'app' in globals():
                print("   ✅ CrossFit 앱: 초기화됨")
                
                # 주요 컴포넌트 상태
                components = {
                    "사용자 모델": app.user_model is not None,
                    "QA 모델": app.qa_model is not None,
                    "VectorDB 모델": app.vector_db_model is not None,
                    "인증 컨트롤러": app.auth_controller is not None,
                    "챗봇 컨트롤러": app.chatbot_controller is not None
                }
                
                for name, status in components.items():
                    print(f"   {'✅' if status else '❌'} {name}: {'정상' if status else '실패'}")
                
                # 앱 상태 데이터
                if hasattr(app, 'app_state'):
                    glossary_count = len(app.app_state.get('glossary', []))
                    evidence_count = len(app.app_state.get('evidence_library', []))
                    print(f"   📚 용어사전: {glossary_count}개")
                    print(f"   ? 증거자료: {evidence_count}개")
            else:
                print("   ❌ CrossFit 앱: 초기화되지 않음")
                
        except Exception as e:
            print(f"   ❌ 앱 상태 확인 실패: {e}")
        
        # 7. Colab 사용 팁
        if IN_COLAB:
            print("\n💡 Google Colab 사용 팁:")
            print("   🔄 세션이 끊어지면 처음부터 다시 실행하세요")
            print("   💾 중요한 데이터는 Google Drive에 저장하세요")
            print("   ⏰ 무료 버전은 12시간 제한이 있습니다")
            print("   🌐 share=True로 외부 접속 가능한 링크를 생성합니다")
            
    except Exception as e:
        print(f"❌ Colab 시스템 진단 실패: {e}")

# Google Colab용 진단 실행
colab_system_diagnostics()

In [ ]:
# Google Colab용 최종 요약 및 사용 가이드
def colab_final_summary_and_guide():
    """Google Colab용 최종 요약 및 사용 가이드"""
    print("📋 CrossFit 코칭 애플리케이션 - Google Colab 가이드")
    print("=" * 60)
    
    if IN_COLAB:
        print("\n? Google Colab 특별 안내:")
        print("   ? 현재 Google Colab에서 실행 중입니다")
        print("   ? share=True로 외부 접속 링크를 제공합니다")
        print("   💾 세션 종료 시 데이터가 사라질 수 있습니다")
        print("   ⏰ 무료 계정은 12시간 사용 제한이 있습니다")
    
    print("\n🏗️ 시스템 아키텍처:")
    print("   ? MVC 구조 기반 모듈화 설계")
    print("   🗄️ SQLite + ChromaDB 하이브리드 데이터베이스")
    print("   🤖 OpenAI GPT-4o-mini 통합 AI 시스템")
    print("   ?️ Gradio 기반 웹 인터페이스")
    
    print("\n🎯 주요 기능:")
    print("   1️⃣ 🔐 사용자 인증 시스템 (로그인/회원가입)")
    print("   2️⃣ 🤖 AI 챗봇 (질문답변, 용어 검색)")
    print("   3️⃣ 📚 크로스핏 용어사전 (23개 주요 용어)")
    print("   4️⃣ 🎯 개인 맞춤 운동 추천")
    print("   5️⃣ 📹 비디오 코칭 및 자세 분석")
    print("   6️⃣ 📊 주제별 자동 응답 생성")
    print("   7️⃣ 🛠️ 관리자 VectorDB 관리")
    
    if IN_COLAB:
        print("\n🚀 Google Colab 실행 방법:")
        print("   1️⃣ Google Drive 마운트 (자동 실행됨)")
        print("   2️⃣ API 키 설정 (Colab 비밀 섹션에서)")
        print("   3️⃣ 프로젝트 업로드 (/content/drive/MyDrive/2조_3rd_프로젝트)")
        print("   4️⃣ 모든 셀 실행 (Ctrl+F9)")
        print("   5️⃣ Gradio 링크 클릭하여 웹앱 접속")
        
        print("\n? API 키 설정 방법:")
        print("   1️⃣ Colab 왼쪽 사이드바에서 🔑 아이콘 클릭")
        print("   2️⃣ 'Add new secret' 버튼 클릭")
        print("   3️⃣ Name: OPENAI_API_KEY, Value: 실제 API 키")
        print("   4️⃣ (선택) Name: LANGCHAIN_API_KEY, Value: 실제 API 키")
        print("   5️⃣ 노트북 액세스 권한 허용")
        
        print("\n📁 Google Drive 구조:")
        print("   Google Drive/")
        print("   └── MyDrive/")
        print("       └── 2조_3rd_프로젝트/")
        print("           ├── src/")
        print("           ├── data/")
        print("           ├── notebooks/")
        print("           ├── requirements.txt")
        print("           └── run.py")
    else:
        print("\n💻 로컬 실행:")
        print("   터미널: python run.py")
        print("   웹: http://localhost:7860")
    
    print("\n🛠️ 문제 해결 (Google Colab):")
    print("   1️⃣ 'Module not found' 오류")
    print("      → Google Drive 마운트 및 프로젝트 경로 확인")
    print("   2️⃣ 'API key not found' 오류")
    print("      → Colab 비밀 섹션에서 API 키 설정")
    print("   3️⃣ '403 Forbidden' 오류")
    print("      → 노트북 액세스 권한 확인")
    print("   4️⃣ 패키지 설치 실패")
    print("      → 런타임 다시 시작 후 재실행")
    print("   5️⃣ 메모리 부족 오류")
    print("      → 런타임 유형을 GPU로 변경")
    
    print("\n? Google Colab 사용 팁:")
    print("   ? 중요한 데이터는 Google Drive에 저장")
    print("   ? 세션 끊김 시 처음부터 다시 실행")
    print("   ⏰ 12시간 후 자동 세션 종료 (무료 계정)")
    print("   ? share=True로 다른 사람과 링크 공유 가능")
    print("   💾 Pro 계정 사용 시 더 많은 리소스 이용")
    
    print("\n🎉 실행 순서 (Google Colab):")
    print("   1️⃣ 첫 번째 셀: Drive 마운트 및 환경 설정")
    print("   2️⃣ 두 번째 셀: 패키지 설치 및 모듈 import")
    print("   3️⃣ 세 번째 셀: Colab 설정 확인")
    print("   4️⃣ 네 번째 셀: 애플리케이션 초기화")
    print("   5️⃣ 테스트 셀들: 각 기능 테스트 (선택사항)")
    print("   6️⃣ Gradio 실행 셀: 웹 애플리케이션 시작")
    
    # 현재 상태 요약
    try:
        if 'app' in globals():
            models_ok = all([
                app.user_model is not None,
                app.qa_model is not None,
                app.vector_db_model is not None
            ])
            
            controllers_ok = all([
                app.auth_controller is not None,
                app.chatbot_controller is not None,
                app.video_controller is not None,
                app.recommendation_controller is not None,
                app.topic_controller is not None,
                app.admin_controller is not None
            ])
            
            print(f"\n🎯 현재 시스템 상태:")
            print(f"   📊 모델: {'✅ 정상' if models_ok else '❌ 문제 있음'}")
            print(f"   🎮 컨트롤러: {'✅ 정상' if controllers_ok else '❌ 문제 있음'}")
            print(f"   🗄️ 용어사전: {len(app.app_state.get('glossary', []))}개 용어")
            print(f"   📚 증거 자료: {len(app.app_state.get('evidence_library', []))}개 문서")
            
            if IN_COLAB:
                drive_status = "✅ 연결됨" if os.path.exists("/content/drive") else "❌ 연결 안됨"
                api_status = "✅ 설정됨" if 'OPENAI_API_KEY' in os.environ else "❌ 설정 안됨"
                print(f"   📁 Google Drive: {drive_status}")
                print(f"   🔑 OpenAI API: {api_status}")
            
            if models_ok and controllers_ok:
                print("   🚀 Google Colab에서 애플리케이션이 준비되었습니다!")
                if IN_COLAB:
                    print("   🌐 Gradio 셀을 실행하여 웹앱을 시작하세요!")
            else:
                print("   ⚠️  일부 컴포넌트에 문제가 있습니다.")
        else:
            print(f"\n⚠️  애플리케이션이 초기화되지 않았습니다.")
            print("   💡 애플리케이션 초기화 셀을 먼저 실행하세요.")
            
    except Exception as e:
        print(f"\n❌ 상태 확인 실패: {e}")
    
    print("\n" + "=" * 60)
    print("🎉 Google Colab에서 CrossFit 코칭 애플리케이션을 즐겨보세요!")

# Google Colab용 최종 요약 실행
colab_final_summary_and_guide()

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e12e22dc60478744c2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7920 <> https://e12e22dc60478744c2.gradio.live
Gradio 앱이 포트 7920에서 실행 중입니다. Public 링크를 확인해주세요.
